# Statistical birds eye view of the contents in an AiiDAdb

This is the first of two deliverable for the SiSc-Lab2020 project.

Authors = 

Supervisors: Dr. Jens Bröder, Dr. Daniel Wortmann, Johannes Wasmer, Prof. Dr. Stefan Blügel.

# Instructions by supervisors

## Jens

You have to implement this notebook.

In the end only text (markdown) cells and output results of code cells should be seen if one hides the code cells (hide_code extension).

That can easily exported into a nice pdf file (google it, probably will find sth with `nbconvert`).

Also the amount of python code in this notebook should be minimal.

Rather, export the functions you use to python file(s) and import them here (hide complexity).

Optional dump query results in a file, from which results will be reread for speed, i.e cache results.

## Johannes



After talking with Jens about it, here are some additional thoughts on the code structure and implementation, for both deliverables.

The **primary** goal is of course that the code should work, produce nice output (and helpful error messages), obviously.

The **secondary** goal is speed. How long do you expect your code to run on a dataset of a given size? Are there multiple paths to a goal, but with differing performance?

You can break the runtime down into several steps: data acquisition, data transformation (or preprocessing), data analysis, data visualization. In this project, we will rename/replace these steps to: **querying, de-/serialization, analysis=visualization**.

**Querying the database.** Performance considerations:
- Performance measurement: use the magics `%time` and `%timeit`.
- Query evaluations: queries (in general) use 'lazy evaluation'.
  - *Query building* methods build the query but do not execute it. These are chainable methods like `append()`, `get_outgoing()`, etc.
  - *Query execution* methods send the query to the database to be evaluated. There are two kinds:
    - non-iterator methods: e.g. `all()`, `first()`, etc. These return a result `list`: all items are loaded into memory.
    - iterator methods: e.g. `iterall()`, `iterdict()`. These return a result `Generator`: only one item at a time is loaded into memory.
    
**De-/serialization**, i.e. writing and reading it to/from a file. *Keep in mind: if you come to the conclusion this is unnecessary, then justify it!* Considerations:
- Necessity: we assume 'yes'. So you need serialization/deserialization routine(s).
- Code design: we recommend to write a serializer that moves *all* data needed from aiida to file (perform query & serialization). Then the visualization methods are decoupled from aiida and load data from that file. Advantages: a) only needs to be called when data in database changed, b) similar queries for different visualizations can be performed only once. One design option is this:
  ```python
  serialize = sisclab.Serializer(profile)
  serialize.to_file(filepath)
  visualize = sisclab.Visualizer(filepath)
  visualize.histogram(cumulative=True, plot_options)
  # plots histogram
  ```
- Serialization format: there are two practical options (maybe more):
  - `dict`: tree-like. JSON format. One `dict` per file. choose key-value (nested?) based on use-case. in general, `uuid` is a good key.
  - `pandas.Dataframe`: could be preferrable in some cases.
- Serialization location:
  - one file or several files?
  - we recommend to de/serialize from/to `sisclab/data/` folder. It is included in the project's `.gitignore` file, so nothing in it gets committed to/from git (git is for code, not for data; the code generates the data).
- Transformation:
  - if needed, decide where to put needed data transformations (before serialization or after deserialization) to minimize them.
- Deserialization: 
  - a class (as above) might help to define the deserialization format only once for all visualization methods.
  

**Visualization**:
- Prefer `bokeh` to `matplotlib` or other libs wherever possible, unless you have a good justification.
- In `D1`, static plots are okay, interactive plots are a bonus.
- Lists results (when plot is overkill) will look nicer in a notebook if they are a `pandas.Series` or `pandas.Dataframes`.
- Think about function signatures. Can you generalize them to make a nice interface? For example, a signature for SubtaskD1.c might look like this:
  ```python
  def node_type_summary(user_list : list = [], node_basetype : Node = Data,
                        chart_type : bokeh.chart_type = bokeh.pie_chart, plot : bool = True):
    """
    :param user_list: list of users. empty list = all users = default.
    :param node_basetype: subdivides chart into subtypes. Valid base type examples: ProcessNode, CalculationNode, WorkflowNode, Data, ArrayData.
    :param chart_type: bokeh visualization type. pie chart = default.
    :param plot: True: show plot, don't return data. False: don't plot, return data.
    :return: stats: a dictionary {node_subtype : node_count}, insertion-order sorted in descending order.
    :rtype: dict.
    """
  ```


# Imports

In [1]:
# magics:
# # autoreload imports. 
# # intent: if i change sth in import, i don't have to restart kernel. enable only for development.
%load_ext autoreload
%autoreload 2
# # choose matplotlib backend. backend 'notebook' allows interactive plots if your env allows it.
%matplotlib notebook


In [2]:
# python imports:
from collections import Counter
import time
#from pprint import pprint

In [3]:
#%pylab inline
#figuresize=(18, 4)
from collections import Counter
from math import pi
import pandas as pd
from bokeh.io import output_file,output_notebook, show
from bokeh.layouts import column
from bokeh.palettes import Category20c
from bokeh.plotting import figure
from bokeh.transform import cumsum
from bokeh.models import Legend,LegendItem

In [4]:
# aiida imports:
from aiida import load_profile
profile = load_profile()

# ggf add futher imports
from aiida.orm import QueryBuilder as QB
from aiida.orm import Node, User, CalcJobNode, Computer, Code
from aiida.plugins import DataFactory

from aiida.common.constants import elements as PeriodicTableElements

In [5]:
# project imports:
import helpers

# equivalent ('.' is the sisc_lab directory):
# from . import helpers
# alternative:
# from .helpers import print_bold
# from .helpers import * ('*' import everything; use of '*' is considered bad style)

In [6]:
# (example:)
helpers.print_bold(f"This notebook/dashboard will visualize the contents from the database of profile {profile.name}")

This notebook/dashboard will visualize the contents from the database of profile generic


# SubtaskD1.a: Node information

Task:

In [7]:
# query for all nodes
print('Information on nodes in the DB: \n')
now = time.strftime("%c")
print('last executed on {}'.format(now))
q = QB()
q.append(Node, project=['id', 'ctime', 'mtime', 'node_type'], tag='node')
q.append(User, with_node='node', project='email')
# TODO: execute query here
t = time.time()
elapsed = time.time() - t
res = q.all()
totalnodes = len(res)
print("Total number of nodes in the database: {} (retrieved in {} s.)".format(totalnodes, elapsed))

Information on nodes in the DB: 

last executed on Wed Jan  6 12:05:19 2021
Total number of nodes in the database: 7792 (retrieved in 0.00016808509826660156 s.)


# SubtaskD1.b: Users

Task: print out a list of Users and how many nodes belong to them

for example

```
Users:
- j.broeder@fz-juelich.de created 182 nodes
- tests@aiida.mail created 104 nodes
```

In [13]:
users = Counter([r[4] for r in res])
print("Users:")
for count, email in sorted((v, k) for k, v in users.items())[::-1]:
    print("* {} created {} nodes".format(email, count))

Users:
* j.broeder@fz-juelich.de created 5629 nodes
* nicolas.mounet@epfl.ch created 1981 nodes
* gianluca.prandini@epfl.ch created 93 nodes
* aiida@localhost created 86 nodes
* antimo.marrazzo@epfl.ch created 3 nodes


# SubtaskD1.c: Node types

Task: plot node information in two pie chart plots

One showing what data nodes there (with their lowest class names(node_type)) I.e Dict, K-pointsData, CifData, FleurinpData...

And one chart showning the process nodes, (with their lowest class names(process_type) i.e CalcjobNodes: FleurCalcjob, FleurinputgenCalcjob, ...

WorkChain nodes: FleurSCFWorkchain, FleurBandDosWorkchain, ..., calcfunctions, and workfunction nodes are fine to not show the lowest class names


In [14]:
#node types
types = Counter([r[3] for r in res])
print("Node types:")

for count, typestring in sorted((v, k) for k, v in types.items())[::-1]:
    print("* {}: {} nodes".format(typestring, count))

Node types:
* data.structure.StructureData.: 5511 nodes
* data.dict.Dict.: 707 nodes
* data.array.kpoints.KpointsData.: 342 nodes
* data.upf.UpfData.: 264 nodes
* process.calculation.calcjob.CalcJobNode.: 214 nodes
* data.remote.RemoteData.: 214 nodes
* data.folder.FolderData.: 214 nodes
* data.array.ArrayData.: 171 nodes
* process.workflow.workfunction.WorkFunctionNode.: 49 nodes
* data.fleur.fleurinp.FleurinpData.: 42 nodes
* data.code.Code.: 36 nodes
* process.workflow.workchain.WorkChainNode.: 28 nodes


In [15]:
#split data nodes and process nodes
labelst_1,labelst_2=[],[]
sizest_1,sizest_2=[],[]
#labelst = [label.split('.')[0]=='data' for label in types.keys()]
#sizest = [nnodes for nnodes in types.values()]
for k,v in types.items():
    if k.split('.')[0]=='data':
        labelst_1.append(k.split('.')[-2])
        sizest_1.append(v)
    elif k.split('.')[0]=='process':
        labelst_2.append(k.split('.')[-2])
        sizest_2.append(v)
        
#plot data nodes
#output_file("pie.html")
output_notebook()
x = dict(zip(labelst_1,sizest_1)) 
data=pd.DataFrame.from_dict(dict(x),orient='index').reset_index().rename(index=str,columns={0:'value','index':'data_nodes'})
data['angle'] = data['value']/sum(x.values()) * 2*pi
data['color'] = Category20c[len(x)]
p = figure(plot_height=350, title="Data Nodes", toolbar_location=None,
           tools="hover", tooltips="@data_nodes: @value")
p.wedge(x=0, y=1, radius=0.4,
        start_angle=cumsum('angle', include_zero=True), end_angle=cumsum('angle'),
        line_color="white", fill_color='color', legend_field='data_nodes', source=data)
p.axis.axis_label=None
p.axis.visible=False
p.grid.grid_line_color = None 
#show(p)                        

#plot process node

x1 = dict(zip(labelst_2,sizest_2)) 
data=pd.DataFrame.from_dict(dict(x1),orient='index').reset_index().rename(index=str,columns={0:'value','index':'process_nodes'})
data['angle'] = data['value']/sum(list(x1.values())) * 2*pi
data['color'] = Category20c[len(x1)]
p1 = figure(plot_height=350, title="Process Nodes", toolbar_location=None,
           tools="hover", tooltips="@process_nodes: @value")
p1.wedge(x=0, y=1, radius=0.4,
        start_angle=cumsum('angle', include_zero=True), end_angle=cumsum('angle'),
        line_color="white", fill_color='color', legend_field='process_nodes', source=data)
p1.axis.axis_label=None
p1.axis.visible=False
p1.grid.grid_line_color = None 
show(column(p,p1))


Loading BokehJS ...

# SubtaskD1.d: Histogram

Task: Cumulative Histogram/ or line plot by ctime & mtime of all nodes over time

In [31]:
# line plot by ctime & mtime

ctimes = sorted(r[1] for r in res)
mtimes = sorted(r[2] for r in res)
num_nodes_integrated = range(len(ctimes))
df = pd.DataFrame({'A':ctimes,"B":mtimes})

#print(df.head())
#df = pd.DataFrame({'A':np.random.randn(100).cumsum(),"B":np.random.randn(100).cumsum()})

#plot multiline
p = figure(plot_width=900, plot_height=300, x_axis_type='datetime')
r=p.multi_line([df['A'], df['B']],  
               [df.index, df.index],   
               color=["firebrick", "navy"],   
               alpha=[0.8, 0.6],     
               line_width=[2,1],     
              )

legend=Legend(items=[
    LegendItem(label="ctime",renderers=[r],index=0),
    LegendItem(label="mtime",renderers=[r],index=1),
])
p.add_layout(legend)
p.xaxis.axis_label = 'Date'
p.yaxis.axis_label = 'Number of nodes'

show(p)

# SubtaskD1.e: Codes

Task: List Code names, sorted by by how many calcjobs where run with each

In [86]:
codes = Code.objects.all()
result = {code.label : len(code.get_outgoing(node_class=CalcJobNode).all_nodes()) for code in codes}
result_df=pd.Series(result).sort_values(ascending=False)
result_df

pw-SVN-piz-daint           171
fleur_mpi_v0.28             22
inpgen                      21
yambo-4.5.1-yambo            0
yambo-4.5.1-p2y              0
inpgen_v0.28                 0
inpgen_iff_0.28              0
fleur_iff_0.28               0
fleur_mpi_max_2              0
fleur_dev_mpi                0
fleur_max_2                  0
inpgen_max_2                 0
fleur_dev_mpi_booster        0
fleur_mac_3_mpi              0
fleur_max_1.3_dev            0
fleur_mpi_max_1.3_dev        0
wannier90-3.1.0-wannier      0
cp2k-7.1                     0
siesta-v4.1-rc1-siesta       0
fleur-0.30-inpgen            0
fleur-0.30-fleur_MPI         0
qe-6.5-matdyn                0
qe-6.5-dos                   0
qe-6.5-q2r                   0
qe-6.5-pw2wannier90          0
qe-6.5-projwfc               0
qe-6.5-neb                   0
qe-6.5-ph                    0
qe-6.5-pp                    0
qe-6.5-cp                    0
qe-6.5-pw                    0
fleur_v0.28                  0
dtype: i

# SubtaskD1.f: Groups

Task: List all group names with how many nodes they contain (verdi group list -C) (exclude import and export groups)

In [2]:
def ListGroup(Group : list, exclude : list):
    """ return the group names and nodes they contain
    
    :the list of all groups
    :the list of all excluded groups name
    """
    print('{:<52}{:6}'.format('Group names:','sizes:'))
    for a in Group:
        flag=0
        type = a[0].type_string
        for ex in exclude:
            if ex in type:
                flag=1
        if(flag):
            continue     
        else:
            ## the line below contains all the properties
            ##print(a[0].label,' ',a[0].user,' ',a[0].type_string,' ',a[0].description)

            print('{:<50}|{:5}'.format(a[0].label,len(a[0].nodes)))

In [5]:
load_profile()
#!verdi group list --all
from aiida.orm import QueryBuilder
from aiida.orm import load_node, Node, Group, Computer, User, CalcJobNode, Code, StructureData

qb = QueryBuilder()
qb.append(Group)
#print(qb.all())

group = qb.all()
ListGroup(group,['export','import'])

Group names:                                        sizes:
SSSP_1.1_efficiency                               |   85
SSSP_1.1_precision                                |   85
sg15-oncv-1.1                                     |   69
codes                                             |   18
fusion_materials                                  |   79
stable_binary_metals_from_materialsproject        | 5058
fleur_initial_cls_finished                        |    7
delta_parameters_gutstav_soc                      |   71
fusion_strucs_refinded                            |   23


# SubtaskD1.g: Structures

Task: Further analyze what structures are in the DB

Number of structureData node versus how many atoms they contain. 

here interactive with bokeh hover tool showing the structure formula and uuid

Number of StructureData nodes versus elements bokeh bar chart, since there are over 
100 elements in the periodic table you can split it over several plots, or just use the charge number as in 
'example/element_content.png' but then make it interactive that once one hovers 
with the mouse over a bar it tells you what element it is and how many structures there are containing this element-

In [6]:
from aiida.orm import QueryBuilder
from aiida.orm import load_node, Node, Group, Computer, User, CalcJobNode, Code, StructureData
from HelperPackage import DataProcessing
from DataProcessing.DataVisu import StrucDataForm

def NumStructureNode():
    #### return the pd.DataFrame including elements and number of each element
    import pandas as pd
    StrucList = []
    qb = QueryBuilder()
    qb.append(StructureData)
    print('number of StructureData Nodes:',qb.count())
    qb.count()

    for struc, in qb.all()[:]:
        form = struc.get_formula()
        struct = StrucDataForm(form)
        StrucList = StrucList+ [struct.FormAnalyse()]
    return pd.DataFrame(StrucList).fillna(0)

def ShowingElements(Data):
    from bokeh.io import output_file, show
    from bokeh.models import ColumnDataSource
    from bokeh.plotting import figure
    from bokeh.io import output_notebook
    from bokeh.palettes import inferno

    output_file("ShowingElements.html")
    #data = NumStructureNode()
    data = Data
    elements = list(data.columns)
    counts = list(data.astype(bool).sum(axis=0))
    #print(counts)
    #print(elements)
    
    source = ColumnDataSource(data=dict(elements=elements, counts=counts,color=inferno(len(elements))))
    
    TOOLTIPS = [
    ("element", "@elements"),
    ("(x,y)", "($x, $y)"),
    ("Structures containing this element", "@counts"),
    ]

    p = figure( y_range=elements,x_range=(0,500), plot_width=800, plot_height=800, title="Elements Counts", tooltips=TOOLTIPS)
    #print('step figure done')
    p.hbar(y="elements", right="counts", height=0.5, left=0, color='color',  source=source)
    #print('step hbar done')
    
    output_notebook()
    p.xgrid.grid_line_color = None
    #p.legend = False
    show(p)
    
    
def ShowFormula(Data):
    from bokeh.io import output_file, show
    from bokeh.models import ColumnDataSource
    from bokeh.plotting import figure
    from bokeh.io import output_notebook
    from bokeh.palettes import inferno

    data = Data
    elements = list(data.keys())
    counts = list(len(data[key])/2 for key in data.keys())
    formulas = list(data[key] for key in data.keys())

    length = len(elements)
    source = ColumnDataSource(data=dict(elements=elements, counts=counts,color=inferno(length)))
    
    TOOLTIPS = [
    ("Number of Atoms", "@elements"),
    ("(x,y)", "($x, $y)"),
    ("Number of Nodes", "@counts"),
    ("id and formula", "@formulas"),
    ]

    p = figure( y_range=(0,160),x_range=(0,1000), plot_width=800, plot_height=800, title="Atoms Count", tooltips=TOOLTIPS)
    #print('step figure done')
    p.hbar(y="elements", right="counts", height=0.5, left=0, color='color',  source=source)
    #print('step hbar done')
    
    output_notebook()
    p.xgrid.grid_line_color = None
    #p.legend = False
    show(p)
    
def GetFormulaDict():
    
    ## may be not necessary, return the formulas and nodes containing this formula
    import numpy as np
    from aiida.orm import QueryBuilder,StructureData
    qb = QueryBuilder()
    qb.append(StructureData)
    Mydict = qb.all()
    Mydict = np.ravel(Mydict)

    Newdict = {}
    for dict in Mydict:
        if dict.get_formula in Newdict.keys():
            Newdict[dict.get_formula()].append(dict.uuid)
        else:
            Newdict[dict.get_formula()] = [dict.uuid]
    return Newdict

def AtomsNumNodes():
    from aiida.orm import QueryBuilder,StructureData
    import numpy as np
    qb = QueryBuilder()
    qb.append(StructureData)
    StructDatas = qb.all()
    Newdict = {}
    
    for data, in StructDatas:
        CompositionDict = data.get_composition()
        NumAtom = np.sum(list(CompositionDict.values()))
        if NumAtom in Newdict.keys():
            Newdict[NumAtom].append(data.uuid)
            Newdict[NumAtom].append(data.get_formula())
        else:
            Newdict[NumAtom] = [data.uuid,data.get_formula()]
    return Newdict

/home/max/Desktop/sisclab new/aiida-jutools/aiida_jutools/sisc_lab/HelperPackage
__init__.py Initialization successful
/home/max/Desktop/sisclab new/aiida-jutools/aiida_jutools/sisc_lab/HelperPackage
__init__.py Initialization successful


In [7]:
dic = AtomsNumNodes()
ShowFormula(dic)

Loading BokehJS ...

In [8]:
data = NumStructureNode()
ShowingElements(data)

number of StructureData Nodes: 5169


Loading BokehJS ...

In [9]:
# initialize the Structure data
from aiida.orm import QueryBuilder
from aiida.orm import load_node, Node, Group, Computer, User, CalcJobNode, Code, StructureData
from HelperPackage import DataProcessing
from DataProcessing.DataVisu import StrucDataForm

qb = QueryBuilder()
qb.append(StructureData)

structures = qb.all()

for structure in structures[:]:
    formula = structure[0].get_formula()
    struct = StrucDataForm(formula)
    structure[0].get_composition()
    print(struct.FormAnalyse())

{'Mg': '16', 'Zn': '80'}
{'Cd': '3', 'Zr': '1'}
{'Nd': '2', 'Te': '4'}
{'As': '4', 'Np': '2'}
{'Co': '2', 'O': '2'}
{'Ba': '10', 'Pb': '6'}
{'Ca': '2', 'Sb': '4'}
{'As': '24', 'Nb': '40'}
{'S': '8', 'V': '4'}
{'B': '20', 'Gd': '8'}
{'Co': '2', 'Y': '2'}
{'O': '2', 'Ti': '12'}
{'Na': '4', 'O': '8'}
{'Se': '40', 'V': '23'}
{'Cd': '2', 'Li': '2'}
{'Fe': '5', 'Y': '1'}
{'C': '12', 'Mn': '28'}
{'Lu': '10', 'Rh': '6'}
{'Ag': '2', 'Nd': '1'}
{'S': '32', 'Tb': '22'}
{'Cu': '2', 'Te': '2'}
{'Cl': '24', 'Mo': '6'}
{'Pu': '1', 'Si': '2'}
{'As': '12', 'V': '16'}
{'Pb': '6', 'Zr': '10'}
{'Au': '2', 'Nb': '1'}
{'Hf': '5', 'Sc': '1'}
{'Fe': '4', 'Sb': '12'}
{'Cu': '6', 'Pm': '2'}
{'In': '3', 'Mg': '1'}
{'Eu': '2', 'Pb': '2'}
{'K': '16', 'Tl': '22'}
{'Pt': '5', 'Sr': '1'}
{'Ag': '20', 'Sr': '20'}
{'Fe': '29', 'Nd': '3'}
{'In': '2', 'Sc': '6'}
{'Ru': '6', 'V': '2'}
{'Hf': '1', 'Zn': '1'}
{'K': '4', 'O': '8'}
{'Lu': '1', 'S': '1'}
{'Al': '4', 'Nd': '2'}
{'O': '3', 'Pa': '1'}
{'Re': '8', 'U': '4'}
{'U': 

{'Re': '6', 'Ru': '2'}
{'Ba': '2', 'Cu': '26'}
{'Ag': '4', 'Eu': '6'}
{'Cd': '2', 'Th': '1'}
{'P': '10', 'Pr': '2'}
{'Na': '6', 'Pt': '2'}
{'Cd': '1', 'Zr': '2'}
{'Be': '2'}
{'Dy': '3', 'Sm': '1'}
{'Nb': '1', 'S': '2'}
{'Gd': '1', 'P': '1'}
{'Co': '17', 'Th': '2'}
{'In': '1', 'Pd': '1'}
{'Si': '2', 'Th': '1'}
{'Co': '4', 'S': '8'}
{'Dy': '4', 'Zn': '12'}
{'Cd': '6', 'Y': '2'}
{'Ag': '4', 'Ba': '4'}
{'S': '16', 'V': '10'}
{'In': '1', 'Sm': '1'}
{'Na': '1', 'Rb': '3'}
{'Ca': '2', 'Ge': '4'}
{'Cs': '2', 'Sn': '6'}
{'Ag': '2', 'Be': '4'}
{'In': '3', 'Pr': '1'}
{'B': '6', 'K': '1'}
{'B': '2', 'Pt': '2'}
{'Cr': '6', 'N': '8'}
{'Hg': '1', 'Rb': '3'}
{'O': '8', 'W': '3'}
{'Ge': '8', 'Tb': '4'}
{'Ni': '16', 'Sn': '16'}
{'Al': '3', 'Nb': '1'}
{'Ca': '2', 'Si': '4'}
{'Eu': '1', 'Ge': '2'}
{'O': '49', 'W': '18'}
{'Ga': '2', 'Pt': '6'}
{'Ta': '2', 'V': '4'}
{'Ni': '4', 'Se': '8'}
{'Co': '4', 'Ta': '2'}
{'Pb': '8', 'Pt': '2'}
{'C': '1', 'V': '2'}
{'P': '8', 'Ti': '24'}
{'Ir': '2', 'Ti': '2'}
{'B': '

{'La': '3', 'Pb': '1'}
{'Hf': '4', 'Si': '4'}
{'C': '5', 'Nb': '6'}
{'Mn': '4', 'Zr': '2'}
{'La': '8', 'Sb': '6'}
{'O': '1', 'Ti': '2'}
{'La': '6', 'Rh': '18'}
{'Cd': '2', 'Er': '1'}
{'As': '14', 'Re': '6'}
{'Cr': '2', 'P': '8'}
{'As': '1', 'Y': '1'}
{'Ni': '6', 'P': '3'}
{'O': '26', 'Ti': '14'}
{'As': '2', 'Cr': '2'}
{'Ag': '10', 'Cd': '16'}
{'Bi': '4', 'Cs': '2'}
{'Al': '44', 'Mo': '10'}
{'Pd': '2', 'Tb': '2'}
{'Gd': '1', 'Zn': '1'}
{'Hf': '1', 'Rh': '3'}
{'Mg': '76', 'Sr': '18'}
{'Tb': '1', 'Tl': '3'}
{'Br': '6', 'Np': '2'}
{'C': '1', 'Tb': '2'}
{'Ga': '8', 'Pt': '16'}
{'B': '16', 'Sm': '4'}
{'Ge': '4', 'Hf': '6'}
{'Ir': '2', 'Tc': '6'}
{'In': '1', 'Yb': '3'}
{'Pt': '11', 'Zr': '9'}
{'H': '3', 'Np': '1'}
{'Be': '2', 'Mn': '6'}
{'K': '1', 'Rb': '3'}
{'Mo': '1', 'Ni': '2'}
{'O': '15', 'Sc': '16'}
{'H': '3', 'Pm': '1'}
{'Rh': '3', 'Te': '4'}
{'Tb': '2', 'Zn': '17'}
{'N': '4', 'Pd': '2'}
{'Ru': '8', 'Sn': '12'}
{'Ca': '2', 'Zn': '4'}
{'P': '24', 'Ta': '40'}
{'Sc': '1', 'Zn': '1'}
{'Ag':

{'Be': '27', 'Ti': '9'}
{'Co': '2', 'Zr': '4'}
{'O': '2', 'Pu': '1'}
{'Hg': '6', 'Tb': '2'}
{'Mg': '1', 'Ni': '1'}
{'Y': '2'}
{'Ag': '2', 'Yb': '1'}
{'Ni': '6', 'Zr': '2'}
{'Ga': '6', 'Hf': '10'}
{'Nb': '8', 'P': '20'}
{'Ir': '4', 'Nd': '2'}
{'Dy': '1', 'Pt': '3'}
{'Se': '2', 'V': '2'}
{'Fe': '1', 'Re': '1'}
{'Ga': '2', 'Lu': '6'}
{'B': '4', 'Mn': '4'}
{'S': '4', 'V': '4'}
{'Tb': '2', 'Zn': '4'}
{'Gd': '4', 'Ni': '14'}
{'Hf': '6', 'Si': '4'}
{'N': '1', 'U': '1'}
{'Mo': '1', 'Pt': '2'}
{'B': '4', 'Ta': '3'}
{'Au': '2', 'Na': '4'}
{'O': '44', 'Ti': '26'}
{'O': '8', 'Pb': '4'}
{'In': '8', 'Sr': '8'}
{'B': '84', 'O': '11'}
{'O': '2', 'Sm': '4'}
{'Ag': '6', 'Cd': '2'}
{'Pd': '12', 'Zr': '4'}
{'In': '2', 'Li': '4'}
{'Ni': '6', 'Sn': '2'}
{'Ba': '4', 'Si': '8'}
{'Ba': '4', 'Sb': '2'}
{'Al': '1', 'B': '2'}
{'Au': '3', 'K': '2'}
{'As': '2', 'Ni': '6'}
{'Ni': '9', 'Y': '3'}
{'In': '1', 'Zr': '3'}
{'Al': '8', 'Tm': '12'}
{'Ni': '4', 'P': '16'}
{'Eu': '4', 'F': '12'}
{'Co': '2', 'Th': '2'}
{'O': '

{'Ba': '2', 'Pd': '4'}
{'Cl': '30', 'W': '6'}
{'Cd': '2', 'Y': '1'}
{'Dy': '2', 'Te': '4'}
{'Mn': '1', 'Pt': '3'}
{'H': '3', 'Sc': '1'}
{'Al': '1', 'Ru': '1'}
{'Ga': '4', 'Pu': '4'}
{'Bi': '4', 'Pd': '10'}
{'In': '6', 'Na': '2'}
{'Pb': '4', 'Pd': '2'}
{'Rh': '4', 'Y': '2'}
{'B': '6', 'Sm': '1'}
{'Ga': '3', 'Nb': '1'}
{'Te': '1', 'U': '1'}
{'Be': '17', 'Ti': '2'}
{'Ir': '9', 'Te': '24'}
{'Cr': '4', 'S': '6'}
{'Ni': '2', 'Ti': '2'}
{'Mg': '23', 'Sr': '6'}
{'Pt': '1', 'Te': '2'}
{'Al': '4', 'Pt': '12'}
{'Al': '12', 'Re': '1'}
{'Pb': '3', 'Yb': '1'}
{'As': '8', 'Eu': '6'}
{'C': '2', 'W': '4'}
{'Ir': '6', 'Re': '2'}
{'Ba': '4', 'In': '4'}
{'In': '2', 'Pt': '1'}
{'Pd': '3', 'Sn': '1'}
{'Nb': '8', 'Se': '4'}
{'Al': '3', 'Sc': '1'}
{'Mg': '5', 'Sc': '1'}
{'B': '6', 'Ta': '5'}
{'As': '5', 'Fe': '12'}
{'Au': '6', 'Ce': '2'}
{'Ag': '1', 'Yb': '1'}
{'In': '9', 'Pt': '13'}
{'Al': '4', 'La': '1'}
{'Cu': '24', 'La': '4'}
{'Ge': '2', 'Ho': '2'}
{'Ge': '6', 'Sr': '10'}
{'Cu': '2', 'Se': '2'}
{'Cr': '4'

...

{'Ca': '16', 'Hg': '36'}
{'Cr': '16', 'Ti': '8'}
{'Fe': '9', 'Ga': '12'}
{'Sc': '1', 'Se': '1'}
{'Ta': '22'}
{'As': '6', 'V': '10'}
{'Hg': '4', 'Mg': '8'}
{'Pd': '2', 'Th': '4'}
{'Pb': '1', 'Pr': '3'}
{'Sb': '1', 'Y': '1'}
{'Pd': '12', 'Th': '4'}
{'Nb': '1', 'Ru': '3'}
{'Se': '2', 'Ti': '2'}
{'Cu': '5', 'Ho': '1'}
{'Ru': '6', 'Th': '14'}
{'O': '4', 'Pd': '2'}
{'Al': '4', 'Ba': '1'}
{'Mn': '4', 'U': '2'}
{'Ag': '6', 'Ce': '2'}
{'Nb': '1', 'Rh': '3'}
{'Pt': '2', 'Th': '2'}
{'Pd': '3', 'Sm': '1'}
{'Fe': '3', 'Pt': '1'}
{'Hg': '6', 'La': '2'}
{'Au': '2', 'Nb': '6'}
{'Ba': '2', 'Zn': '1'}
{'C': '4', 'Ti': '8'}
{'Co': '2', 'Sb': '2'}
{'Cu': '4', 'Ti': '3'}
{'Mg': '2', 'Zn': '4'}
{'Te': '20', 'U': '4'}
{'Pd': '3', 'Tm': '1'}
{'Fe': '24', 'N': '8'}
{'Nd': '2', 'Ni': '4'}
{'Se': '4', 'Ta': '2'}
{'Au': '2', 'Tl': '4'}
{'Er': '1', 'Tl': '1'}
{'Be': '26', 'Ce': '2'}
{'Hf': '14', 'P': '8'}
{'Au': '1', 'Y': '1'}
{'Tb': '1', 'Te': '1'}
{'Ni': '2', 'Sb': '2'}
{'Cd': '72', 'Yb': '12'}
{'Fe': '7', 'W': 

{'Hf': '2', 'Pt': '2'}
{'Pd': '1', 'Zr': '1'}
{'Co': '34', 'Pu': '4'}
{'N': '1', 'Nd': '1'}
{'Tb': '4', 'Zn': '34'}
{'B': '6', 'Ru': '14'}
{'Ba': '2', 'Zn': '26'}
{'O': '26', 'V': '12'}
{'Nb': '1', 'Ru': '1'}
{'Ni': '8', 'P': '8'}
{'Cr': '2', 'S': '4'}
{'Ca': '4', 'Mg': '8'}
{'Ce': '10', 'Ga': '6'}
{'H': '3', 'La': '1'}
{'As': '2', 'Ti': '2'}
{'Cl': '36', 'Pr': '15'}
{'Ce': '6', 'Ni': '18'}
{'B': '16', 'Pr': '4'}
{'Mn': '40', 'Si': '16'}
{'Ce': '6', 'S': '8'}
{'Sn': '3', 'Th': '1'}
{'P': '4', 'W': '2'}
{'Ca': '2', 'Nd': '6'}
{'P': '1', 'Pu': '1'}
{'Ca': '2', 'Si': '2'}
{'Fe': '1', 'Nb': '3'}
{'Ni': '5', 'Yb': '1'}
{'Mg': '6', 'Na': '2'}
{'Cr': '6', 'N': '8'}
{'Li': '4', 'Mg': '2'}
{'Ni': '12', 'Ta': '4'}
{'Nb': '2', 'Sb': '4'}
{'P': '4', 'Ru': '4'}
{'P': '4', 'Re': '3'}
{'Pr': '1', 'S': '1'}
{'Er': '10', 'Si': '6'}
{'Os': '6', 'Tc': '2'}
{'Si': '3', 'U': '1'}
{'Dy': '6', 'Ge': '10'}
{'Cd': '2', 'Sm': '6'}
{'Ce': '1', 'H': '2'}
{'Ho': '6', 'In': '2'}
{'Ag': '1', 'Mg': '1'}
{'Cd': '3', '

{'Nb': '2', 'Se': '4'}
{'Ge': '3', 'Np': '1'}
{'Mn': '2', 'Ru': '6'}
{'La': '8', 'Te': '16'}
{'Mg': '20', 'Zn': '35'}
{'N': '4', 'Re': '2'}
{'Au': '6', 'Sr': '14'}
{'Nd': '2', 'Pt': '4'}
{'Se': '8', 'Tb': '6'}
{'Al': '3', 'V': '1'}
{'In': '14', 'Pd': '6'}
{'Mg': '2', 'Yb': '4'}
{'Al': '12', 'Mg': '17'}
{'Se': '1', 'Sm': '1'}
{'C': '1', 'Pa': '1'}
{'Hf': '4', 'Si': '2'}
{'Fe': '1', 'Pd': '3'}
{'Cd': '2', 'Tb': '1'}
{'C': '12', 'H': '2'}
{'Dy': '3', 'Y': '1'}
{'Sn': '3', 'Tb': '1'}
{'Co': '2', 'F': '6'}
{'Ir': '3', 'Nb': '1'}
{'Si': '4', 'Ti': '2'}
{'Er': '2', 'Mn': '4'}
{'Ta': '30'}
{'Br': '6', 'U': '2'}
{'Ba': '2', 'Cd': '22'}
{'C': '38', 'Er': '30'}
{'Lu': '2', 'Mn': '10'}
{'Sb': '6', 'Tl': '21'}
{'Pb': '1', 'Pu': '3'}
{'Te': '8', 'U': '6'}
{'Hg': '19', 'Rb': '5'}
{'Ge': '10', 'Tb': '6'}
{'Cd': '1', 'Pr': '3'}
{'Nb': '2', 'Zn': '32'}
{'Rh': '2', 'Ti': '2'}
{'Ge': '16', 'Pr': '20'}
{'O': '4', 'Rb': '18'}
{'Ba': '2', 'Tl': '6'}
{'Pa': '4', 'S': '6'}
{'Tc': '1', 'Ti': '1'}
{'Al': '6', 'Z

{'Ca': '28', 'Ge': '24'}
{'Ga': '12', 'Sm': '20'}
{'Br': '2', 'Ni': '1'}
{'Cd': '1', 'Pr': '1'}
{'Si': '2', 'Tc': '6'}
{'Se': '4', 'Ti': '5'}
{'Pt': '3', 'Tm': '1'}
{'Pt': '1', 'Rh': '3'}
{'Ca': '6', 'Ga': '10'}
{'Ga': '14', 'Li': '6'}
{'Mg': '2', 'Rh': '1'}
{'B': '16', 'Ce': '4'}
{'Li': '1', 'Rh': '3'}
{'Ca': '12', 'Cd': '8'}
{'Ca': '1', 'Ge': '2'}
{'Er': '20', 'Si': '16'}
{'Si': '2', 'Ta': '4'}
{'Ba': '4', 'Bi': '2'}
{'B': '2', 'W': '2'}
{'Ag': '3', 'Sb': '1'}
{'Sc': '2', 'Zn': '6'}
{'In': '2', 'La': '4'}
{'P': '6', 'Pu': '2'}
{'Rh': '1', 'Y': '1'}
{'La': '2', 'Rh': '2'}
{'Mg': '2', 'Tm': '4'}
{'As': '4', 'Mn': '4'}
{'Co': '7', 'Fe': '9'}
{'Sm': '1', 'Zn': '5'}
{'Al': '3', 'U': '1'}
{'Ni': '2', 'Ta': '1'}
{'O': '2', 'Ti': '12'}
{'Hf': '1', 'Pd': '3'}
{'O': '60', 'Ti': '32'}
{'Te': '4', 'Ti': '5'}
{'O': '16', 'W': '6'}
{'Al': '1', 'Re': '2'}
{'Sm': '1', 'Tm': '3'}
{'Ni': '8', 'U': '4'}
{'As': '6', 'Ba': '10'}
{'Ge': '4', 'Pr': '4'}
{'Pt': '2', 'V': '2'}
{'Au': '4', 'Ba': '4'}
{'Al': '

{'Ag': '4', 'Cl': '6'}
{'Fe': '17', 'Gd': '2'}
{'Rh': '4', 'Si': '4'}
{'Ga': '1', 'Sm': '3'}
{'Co': '2', 'Re': '6'}
{'P': '7', 'V': '12'}
{'Si': '4', 'Sm': '2'}
{'Ag': '16', 'Ca': '6'}
{'Ce': '2', 'Te': '6'}
{'Fe': '23', 'Ho': '6'}
{'Cl': '6', 'Zr': '2'}
{'Ir': '3', 'Pa': '1'}
{'Sm': '2', 'Zn': '4'}
{'Nb': '3', 'Ru': '1'}
{'Ce': '1', 'O': '1'}
{'Rh': '3', 'Ti': '1'}
{'C': '12', 'Er': '8'}
{'As': '1', 'Er': '1'}
{'Ce': '2', 'Pt': '2'}
{'Hf': '4', 'P': '4'}
{'Ge': '6', 'Nd': '10'}
{'Ac': '2', 'Au': '6'}
{'Ag': '1', 'Li': '3'}
{'Mn': '8', 'Zr': '4'}
{'Be': '5', 'Sc': '1'}
{'Li': '2', 'Pt': '4'}
{'Pt': '2', 'Te': '2'}
{'Th': '1', 'Tl': '3'}
{'Na': '1', 'Pb': '3'}
{'Mg': '3', 'Ni': '3'}
{'Ac': '1', 'H': '2'}
{'Ga': '10', 'V': '12'}
{'Al': '3', 'Ho': '1'}
{'K': '4', 'Pb': '8'}
{'Pu': '26', 'Zn': '116'}
{'Ag': '36', 'In': '16'}
{'Ga': '3', 'Lu': '1'}
{'Au': '1', 'Er': '1'}
{'As': '6', 'Ca': '10'}
{'Al': '16', 'Cr': '10'}
{'Ba': '10', 'P': '18'}
{'Al': '4', 'Au': '8'}
{'Au': '2', 'Tb': '1'}
{'

# SubtaskD1.h: Calculations

Task: more detail analysis of Calculations

`print('\n\nMore detailed analysis of Calculations \n')`

List, stacked Histogram of Calculations types and the state it ended up finished, failed, exit codes, exit messages

more detail analysis of WorkChains

`print('\n\nMore detailed analysis of WorkChains \n')`

List,  stacked Histogram for each Workchain type and the state it ended up in finished, failed, exit codes, exit messages

# SubtaskD1.i: Provenance

Task: Database and provenance health: display the number of nodes who have no incomming and outgoing links, no incomming links (any number outgoing), and no outgoing links (any number incomming)